In [2]:

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
image_size = 28
num_labels = 10
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [61]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [68]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
#     print('Initialized')
    result = session.run(weights)
    print(result.shape)


(784, 10)


In [74]:
a = np.arange(6).reshape(2,3)
a

array([[0, 1, 2],
       [3, 4, 5]])

In [84]:
"l"=='l'

True

In [63]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.443527
Training accuracy: 8.9%
Validation accuracy: 10.8%
Loss at step 100: 2.242017
Training accuracy: 72.2%
Validation accuracy: 71.0%
Loss at step 200: 1.802279
Training accuracy: 75.2%
Validation accuracy: 73.8%
Loss at step 300: 1.560401
Training accuracy: 76.7%
Validation accuracy: 74.5%
Loss at step 400: 1.397484
Training accuracy: 77.5%
Validation accuracy: 74.7%
Loss at step 500: 1.277084
Training accuracy: 78.1%
Validation accuracy: 74.8%
Loss at step 600: 1.183059
Training accuracy: 79.0%
Validation accuracy: 74.8%
Loss at step 700: 1.106648
Training accuracy: 79.5%
Validation accuracy: 74.9%
Loss at step 800: 1.042768
Training accuracy: 79.8%
Validation accuracy: 74.8%
Test accuracy: 82.7%


In [85]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [86]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.289558
Minibatch accuracy: 7.8%
Validation accuracy: 10.5%
Minibatch loss at step 500: 1.810631
Minibatch accuracy: 70.3%
Validation accuracy: 74.7%
Minibatch loss at step 1000: 1.519584
Minibatch accuracy: 69.5%
Validation accuracy: 76.0%
Minibatch loss at step 1500: 1.566046
Minibatch accuracy: 71.1%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 0.921703
Minibatch accuracy: 78.1%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 1.033332
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 3000: 1.113508
Minibatch accuracy: 74.2%
Validation accuracy: 77.6%
Test accuracy: 85.3%


In [98]:
batch_size = 128
layer1node = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  layer1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1node]))
  layer1_biases = tf.Variable(tf.zeros([layer1node]))
    
  layer2_weights = tf.Variable(tf.truncated_normal(
      [layer1node,num_labels], stddev=0.1))
  layer2_biases = tf.Variable(tf.zeros([num_labels]))
    
   # Model.
  def model(data):
    hidden=tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    return tf.matmul(hidden, layer2_weights) + layer2_biases
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [99]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 41.361710
Minibatch accuracy: 10.2%
Validation accuracy: 16.3%
Minibatch loss at step 500: 26.433386
Minibatch accuracy: 76.6%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 7.083259
Minibatch accuracy: 76.6%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 6.819993
Minibatch accuracy: 71.1%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 5.926030
Minibatch accuracy: 80.5%
Validation accuracy: 80.3%
Minibatch loss at step 2500: 1.867695
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 3000: 4.680519
Minibatch accuracy: 78.9%
Validation accuracy: 80.3%
Test accuracy: 87.8%


In [119]:
batch_size = 128
layer1node = 1024
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. 
  layer1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1node]),name='layer1')
  layer1_biases = tf.Variable(tf.zeros([layer1node]),name='biase1')
    
  layer2_weights = tf.Variable(tf.truncated_normal(
      [layer1node,layer1node], stddev=0.1),name='layer2')
  layer2_biases = tf.Variable(tf.zeros([layer1node]),name='biases2')
    
  layer3_weights = tf.Variable(tf.truncated_normal(
      [layer1node,num_labels]),name='layer3')
  layer3_biases = tf.Variable(tf.zeros([num_labels]),name='bias3')
    
   # Model.
  def model(data):
    hidden1=tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    hidden2=tf.nn.relu(tf.matmul(hidden1, layer2_weights) + layer2_biases)
    return tf.matmul(hidden2, layer3_weights) + layer3_biases
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [120]:
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 765.021606
Minibatch accuracy: 6.2%
Validation accuracy: 21.8%
Minibatch loss at step 500: 1.006157
Minibatch accuracy: 60.2%
Validation accuracy: 66.3%
Minibatch loss at step 1000: 1.016849
Minibatch accuracy: 62.5%
Validation accuracy: 66.9%
Minibatch loss at step 1500: 1.190337
Minibatch accuracy: 68.0%
Validation accuracy: 69.1%
Minibatch loss at step 2000: 0.786865
Minibatch accuracy: 73.4%
Validation accuracy: 71.7%
Minibatch loss at step 2500: 0.934079
Minibatch accuracy: 67.2%
Validation accuracy: 72.4%
Minibatch loss at step 3000: 0.721132
Minibatch accuracy: 71.9%
Validation accuracy: 73.4%
Minibatch loss at step 3500: 1.020341
Minibatch accuracy: 74.2%
Validation accuracy: 74.1%
Minibatch loss at step 4000: 0.645948
Minibatch accuracy: 81.2%
Validation accuracy: 74.1%
Minibatch loss at step 4500: 0.820714
Minibatch accuracy: 75.0%
Validation accuracy: 74.5%
Minibatch loss at step 5000: 0.630560
Minibatch accuracy: 78.9%
Validation accura

In [139]:
batch_size = 128
layer1node = 16
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
    
   # Model.
  def model(data):
    # Variables. 
    layer1_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, layer1node]),name='layer1')
    layer1_biases = tf.Variable(tf.zeros([layer1node]),name='biase1')
     
    layer3_weights = tf.Variable(tf.truncated_normal(
      [layer1node,num_labels]),name='layer3')
    layer3_biases = tf.Variable(tf.zeros([num_labels]),name='bias3')
    hidden=tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
    regularizers = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) 
#     for i in range(10):
#       layermiddle_weights = tf.Variable(tf.truncated_normal(
#            [layer1node,layer1node], stddev=0.1))
#       layermiddle_biases = tf.Variable(tf.zeros([layer1node])
#       hidden = tf.nn.relu(tf.matmul(hidden, layermiddle_weights) + layermiddle_biases)
#       regularizers += (tf.l2_loss(layermiddle_weights) +    tf.l2_loss(layermiddle_biases)  ) 
            
    layermiddle_weights = tf.Variable(tf.truncated_normal(
           [layer1node,layer1node], stddev=0.1))
    layermiddle_biases = tf.Variable(tf.zeros([layer1node])
#     tf.nn.relu(tf.matmul(hidden, layermiddle_weights) + layermiddle_biases)
#     tf.matmul(hidden, layermiddle_weights)
#     regularizers += (tf.l2_loss(layermiddle_weights) +    tf.l2_loss(layermiddle_biases)  ) 
    m= tf.matmul(hidden, layer3_weights) + layer3_biases
    regularizers += (tf.l2_loss(layer3_weights)+tf.l2_loss(layer3_biases))
    return m,regularizers
  # Training computation.
  logits,regularizers = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  loss += 5e-4 * regularizers                                      
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset)[0])
  test_prediction = tf.nn.softmax(model(tf_test_dataset)[0])

SyntaxError: invalid syntax (<ipython-input-139-573acb1f3220>, line 41)

In [122]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 649.322571
Minibatch accuracy: 8.6%
Validation accuracy: 9.9%
Minibatch loss at step 500: 1.186292
Minibatch accuracy: 61.7%
Validation accuracy: 9.9%
Traceback (most recent call last):


ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "/Library/Python/2.7/site-packages/IPython/core/ultratb.py", line 970, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Library/Python/2.7/site-packages/IPython/core/ultratb.py", line 233, in wrapped
    return f(*args, **kwargs)
  File "/Library/Python/2.7/site-packages/IPython/core/ultratb.py", line 267, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1044, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 1004, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/inspect.py", line 454, in getsourcefile
    if hasattr(getmodule(object, filename), '__loa


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable